# Generalized Hasse invariants for $Sp_{2g}$

This simple class allows you to perform some explicit computations with GZipFlags and automorphic line bundles defined over them.

In [1]:
class GZipFlag:
    def __init__(self,g,p):
        self.g = g
        self.p = p
        self.R = RootSystem("A"+str(g-1)).ambient_space() #Ag-1
        self.W =  WeylGroup(self.R,prefix ='s')
        self.RG = RootSystem("C"+str(g)).ambient_space() #Cg
        self.WG = WeylGroup(self.RG, prefix ='s')
        self.P = RootSystem("A"+str(g-1)).weight_lattice()
        self.PG = RootSystem("C"+str(g)).weight_lattice()
        self.phi_G = self.RG.positive_roots()

        I_siegel = [i for i in range(1,g)]
        w0 = self.WG.long_element()
        w0J = self.get_longest_element_connected(I_siegel)
        self.z = w0*w0J
        
    def get_longest_element_connected(self, I):
        simple = self.WG.simple_reflections() 
        res = self.WG.unit()
        for k in range(len(I),0,-1):
            temp = self.WG.unit()
            for j in range(k):
                temp = temp * simple[I[j]]
            res = res * temp
        return res
    
    def Dw(self, w, char):
        char = self.RG(char)
        chi = char - self.p*(self.z*w.inverse()).action(char)
        return chi
    
    def Dw_inverse(self,w,chi):
        chi = self.RG(chi)
        wprime = self.z*w.inverse()
        r = wprime.order()
        char = self.RG(self.g*[0])
        for i in range(r):
            char = char + self.p**i*((wprime**i).action(chi))
        char = -1/(self.p**r-1)*char
        return char
    
    def has_hasse_invariant(self, w, chi, positive_weight = False):
        if positive_weight:
            chiprime = self.z.action(self.RG(chi))
        else:
            chiprime = self.RG(chi)
        char = self.Dw_inverse(w,chiprime)
        char = self.RG(char).to_weight_space()
        values = {}
        has_hasse = True
        for v,alpha in w.bruhat_lower_covers_coroots():
            alpha = self.RG(alpha)
            charprime = w.action(alpha).to_weight_space()
            values[str(v)] = -char.symmetric_form(charprime)
            has_hasse = has_hasse and (values[str(v)] > 0)
        return has_hasse, values
    
    def minimal_left_coset_representatives(self,I):
        simple = self.RG.simple_roots()
        SI = [simple[i] for i in I]
        i = 1
        res = {}
        for w in self.WG:
            is_in = True
            for alpha in SI:
                alpha_prime = w.inverse().action(alpha)
                if alpha_prime not in self.phi_G:
                    is_in = False
                    break
            if is_in:
                res[i] = w
                i = i+1
        return res
    
    def extends_to_I(self, chi, I):
        chi = self.RG(chi).to_weight_space()
        simple = self.RG.simple_coroots()
        SI = [simple[i] for i in I]
        extends = True
        for alpha in SI:
            if chi.symmetric_form(alpha.to_weight_space()) != 0:
                extends = False
                break
        return extends
    
    def has_hasse_invariant_all_strata(self, chi, I, details = False):
        if not self.extends_to_I(chi,I):
            raise ValueError('The character is not defined on the GZipFlag of type I')
        res = True
        I_W = self.minimal_left_coset_representatives(I)
        for w in I_W.values():
            has_hasse, values = self.has_hasse_invariant(w,chi)
            res = res and has_hasse
            if details:
                print(str(has_hasse) + ' : '+ str(w))
        return res

# Example

We define the GZipFlag with $G = Sp_6$ over $\mathbb{F}_7$. 

In [2]:
Y = GZipFlag(g = 3, p = 7)

The next line returns true if the line bundle $\mathcal{L}_{(-2,-3,-5)}$ has generalized Hasse invariants on all strata of the stack $Sp_6-\text{ZipFlag}^{B}$ where $B \subset Sp_6$ is the Borel subgroup.

In [3]:
Y.has_hasse_invariant_all_strata([-2,-3,-5],[])

False

If you want more details on each strata, add "details = True" to the previous command 

In [4]:
Y.has_hasse_invariant_all_strata([-2,-3,-5], [], details = True)

True : 1
True : s3
True : s3*s2
True : s3*s2*s1
True : s3*s2*s3
True : s3*s2*s3*s1
True : s3*s2*s3*s1*s2
True : s3*s2*s3*s1*s2*s3
True : s2
True : s2*s3
True : s2*s3*s2
True : s2*s3*s2*s1
True : s3*s2*s3*s2
True : s3*s2*s3*s2*s1
True : s3*s2*s3*s1*s2*s1
True : s3*s2*s3*s1*s2*s3*s1
True : s2*s1
True : s2*s3*s1
True : s2*s3*s1*s2
True : s2*s3*s1*s2*s1
True : s2*s3*s1*s2*s3
True : s2*s3*s1*s2*s3*s1
True : s2*s3*s1*s2*s3*s1*s2
True : s3*s2*s3*s1*s2*s3*s1*s2
True : s1
True : s3*s1
True : s3*s1*s2
True : s3*s1*s2*s1
True : s3*s1*s2*s3
True : s3*s1*s2*s3*s1
True : s3*s1*s2*s3*s1*s2
True : s3*s2*s3*s1*s2*s3*s2
True : s1*s2
True : s1*s2*s3
False : s1*s2*s3*s2
True : s1*s2*s3*s2*s1
False : s3*s1*s2*s3*s2
True : s3*s1*s2*s3*s2*s1
False : s3*s1*s2*s3*s1*s2*s1
True : s3*s2*s3*s1*s2*s3*s2*s1
True : s1*s2*s1
True : s1*s2*s3*s1
True : s1*s2*s3*s1*s2
False : s1*s2*s3*s1*s2*s1
True : s2*s3*s1*s2*s3*s2
True : s2*s3*s1*s2*s3*s2*s1
True : s2*s3*s1*s2*s3*s1*s2*s1
True : s3*s2*s3*s1*s2*s3*s1*s2*s1


False

Run the next line if you want to know the situation on the particular stratum $w =  s1*s2*s3*s2$ 

In [5]:
w = Y.WG.from_reduced_word([1,2,3,2])
has_hasse, values = Y.has_hasse_invariant(w, [-2,-3,-5])
print(values)
print(has_hasse)

{'s2*s3*s2': 1, 's3*s1*s2': 2/3, 's1*s2*s3': 0}
False


It means that the section $s$ of $\mathcal{L}_{(-2,-3,-5)}$ that lives on the open stratum $w =  s1*s2*s3*s2$ of $Sp_6-\text{ZipFlag}^{B}$ has a divisor $\text{div}(s) = \overline{[s2*s3*s2]} + \frac{2}{3}\overline{[s3*s1*s2]} + 0\overline{[s1*s2*s3]}$. In particular the section $s$ extends to the closed stratum but does not vanish on the border $\overline{[s1*s2*s3]}$. In particular $\mathcal{L}_{(-2,-3,-5)}$ has no Hasse invariant on $w$.

Run the next line to look at the situation of the line bundle $\mathcal{L}_{(-2,-2,-5)}$ on the partial $Sp_6-\text{ZipFlag}^{I}$ of type $I = [(1,-1,0)]$.

In [6]:
Y.has_hasse_invariant_all_strata([-2,-2,-5], [1], details = True)

True : 1
True : s3
True : s3*s2
True : s3*s2*s1
True : s3*s2*s3
True : s3*s2*s3*s1
True : s3*s2*s3*s1*s2
True : s3*s2*s3*s1*s2*s3
True : s2
True : s2*s3
True : s2*s3*s2
True : s2*s3*s2*s1
True : s3*s2*s3*s2
True : s3*s2*s3*s2*s1
True : s3*s2*s3*s1*s2*s1
True : s3*s2*s3*s1*s2*s3*s1
True : s2*s1
True : s2*s3*s1
True : s2*s3*s1*s2
True : s2*s3*s1*s2*s1
True : s2*s3*s1*s2*s3
True : s2*s3*s1*s2*s3*s1
True : s2*s3*s1*s2*s3*s1*s2
True : s3*s2*s3*s1*s2*s3*s1*s2


True

We notice that $\mathcal{L}_{(-2,-2,-5)}$ has generalized hasse invariants for all strata $w \in I_{W} \subset W$.

If we give a character which does not extend to the partial $Sp_6-\text{ZipFlag}^{I}$, we get an error.

In [7]:
Y.has_hasse_invariant_all_strata([-2,-3,-5], [1], details = True)

ValueError: The character is not defined on the GZipFlag of type I